In [ ]:
import pandas as pd
import pandas_datareader as pdr
import yfinance as yf
import datetime

# Define tickers
ticker_2y = "DGS2"  # 2-year Treasury bond yield
ticker_10y = "DGS10"  # 10-year Treasury bond yield
sp500_ticker = "^GSPC"  # S&P 500 Index
cs_ticker = "CSUSHPINSA"  # Case-Shiller Home Price Index

# Set date range
start = datetime.datetime(2000, 1, 1)
end = datetime.datetime(2024, 1, 1)

# Fetch data
data_2y = pdr.DataReader(ticker_2y, 'fred', start, end)
data_10y = pdr.DataReader(ticker_10y, 'fred', start, end)
sp500_data = yf.download(sp500_ticker, start=start, end=end)
cs_data = pdr.DataReader(cs_ticker, 'fred', start, end)

# Data Cleaning and Transformation

# Aligning datasets on the same dates and dropping NaNs
combined_data = pd.concat([data_2y, data_10y], axis=1).dropna()
combined_data.columns = ['2Y', '10Y']
combined_data['Spread'] = combined_data['10Y'] - combined_data['2Y']

# Resample S&P 500 and Case-Shiller data to monthly values
sp500_monthly = sp500_data['Adj Close'].resample('M').mean()
cs_monthly = cs_data.resample('M').mean()

# Standardize date formats
combined_data.index = pd.to_datetime(combined_data.index)
sp500_monthly.index = pd.to_datetime(sp500_monthly.index)
cs_monthly.index = pd.to_datetime(cs_monthly.index)

# Fill missing values
combined_data.fillna(method='ffill', inplace=True)
sp500_monthly.fillna(method='ffill', inplace=True)
cs_monthly.fillna(method='ffill', inplace=True)